# Toronto Neighboorhood Project

<b>PART1<b>

In [1]:
#PULLING TABLE DATA FROM WIKI#


#!pip install html-table-parser-python3
import pandas as pd
import numpy as np
import urllib.request
from pprint import pprint
from html_table_parser import HTMLTableParser


def url_get_contents(url):
    """ Opens a website and read its binary contents (HTTP Response Body) """
    req = urllib.request.Request(url=url)
    f = urllib.request.urlopen(req)
    return f.read()


def main():
    url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
    xhtml = url_get_contents(url).decode('utf-8')

    p = HTMLTableParser()
    p.feed(xhtml)
    global table
    table=p.tables[0]
    pprint(table)
    

if __name__ == '__main__':
    main()

[['Postal Code', 'Borough', 'Neighbourhood'],
 ['M1A', 'Not assigned', 'Not assigned'],
 ['M2A', 'Not assigned', 'Not assigned'],
 ['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village'],
 ['M5A', 'Downtown Toronto', 'Regent Park, Harbourfront'],
 ['M6A', 'North York', 'Lawrence Manor, Lawrence Heights'],
 ['M7A', 'Downtown Toronto', "Queen's Park, Ontario Provincial Government"],
 ['M8A', 'Not assigned', 'Not assigned'],
 ['M9A', 'Etobicoke', 'Islington Avenue, Humber Valley Village'],
 ['M1B', 'Scarborough', 'Malvern, Rouge'],
 ['M2B', 'Not assigned', 'Not assigned'],
 ['M3B', 'North York', 'Don Mills'],
 ['M4B', 'East York', 'Parkview Hill, Woodbine Gardens'],
 ['M5B', 'Downtown Toronto', 'Garden District, Ryerson'],
 ['M6B', 'North York', 'Glencairn'],
 ['M7B', 'Not assigned', 'Not assigned'],
 ['M8B', 'Not assigned', 'Not assigned'],
 ['M9B',
  'Etobicoke',
  'West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale'],
 ['M1C', 'Scarborough', '

In [2]:
tbl=table
features=tbl[0]
print(features)

['Postal Code', 'Borough', 'Neighbourhood']


In [3]:
# delete feature names from table
del tbl[0]
tbl

[['M1A', 'Not assigned', 'Not assigned'],
 ['M2A', 'Not assigned', 'Not assigned'],
 ['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village'],
 ['M5A', 'Downtown Toronto', 'Regent Park, Harbourfront'],
 ['M6A', 'North York', 'Lawrence Manor, Lawrence Heights'],
 ['M7A', 'Downtown Toronto', "Queen's Park, Ontario Provincial Government"],
 ['M8A', 'Not assigned', 'Not assigned'],
 ['M9A', 'Etobicoke', 'Islington Avenue, Humber Valley Village'],
 ['M1B', 'Scarborough', 'Malvern, Rouge'],
 ['M2B', 'Not assigned', 'Not assigned'],
 ['M3B', 'North York', 'Don Mills'],
 ['M4B', 'East York', 'Parkview Hill, Woodbine Gardens'],
 ['M5B', 'Downtown Toronto', 'Garden District, Ryerson'],
 ['M6B', 'North York', 'Glencairn'],
 ['M7B', 'Not assigned', 'Not assigned'],
 ['M8B', 'Not assigned', 'Not assigned'],
 ['M9B',
  'Etobicoke',
  'West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale'],
 ['M1C', 'Scarborough', 'Rouge Hill, Port Union, Highland Creek'],
 ['M

In [4]:
import pandas as pd
import numpy as np
pdTable = pd.DataFrame(data=tbl)
pdTable.columns= features
pdTable.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [5]:
a=pdTable[pdTable.Borough != 'Not assigned']
a=a.reset_index(drop=True)
a.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [6]:
a.shape

(103, 3)

---- end of Part1 ----

<b>PART2<b>

In [7]:
#!pip install pgeocode
import pgeocode

nomi = pgeocode.Nominatim('ca')
print(nomi.query_postal_code("M5G").latitude)
print(nomi.query_postal_code("M5G").longitude)

43.6564
-79.38600000000002


In [19]:
##dummy latitude values , copied fro postal codes
lan=pd.DataFrame(columns=["dummy"])
lan["dummy"]=a["Postal Code"]
lon=pd.DataFrame(columns=["dumy"]) 
lon["dumy"]=a["Postal Code"]

for k in range(len(a['Postal Code'])):
    #print(k)

    lan["dummy"][k]=(nomi.query_postal_code(lan["dummy"][k]).latitude)
    lon["dumy"][k]=(nomi.query_postal_code(lon["dumy"][k]).longitude)

lan["dummy"]
lon["dumy"]


0       -79.33
1     -79.3148
2     -79.3626
3     -79.4504
4     -79.3889
        ...   
98    -79.5076
99     -79.383
100   -79.2505
101   -79.4939
102   -79.5231
Name: dumy, Length: 103, dtype: object

In [23]:
lan.rename(columns={"dummy": "Latitude"})
lon.rename(columns={"dumy": "Longitude"})

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [34]:
a['Latitude'] = lan
a['Longitude'] = lon
a.head(12)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7545,-79.33
1,M4A,North York,Victoria Village,43.7276,-79.3148
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.6662,-79.5282
6,M1B,Scarborough,"Malvern, Rouge",43.8113,-79.193
7,M3B,North York,Don Mills,43.745,-79.359
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.7063,-79.3094
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3783


---- end of Part2 ----

<b>PART3<b>

In [38]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
!pip install folium
import folium # map rendering library

clusterTable=a
# set number of clusters
kclusters = 5

#toronto_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(clusterTable)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

     |████████████████████████████████| 93 kB 3.1 MB/s  eta 0:00:01


ValueError: could not convert string to float: 'Mimico NW, The Queensway West, South of Bloor, Kingsway Park South West, Royal York South West'